# Introduction

In the following notebook, I will be modeling Airbnb Listings data to create a price predictor to using a Deep Learning model

**Read in libraries**

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

**Set notebook preferences**

In [148]:
#Set options for pandas
pd.set_option('display.max_columns',1_000)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#Set options for numpy
np.set_printoptions(suppress=True)

#Set visualization preferences
plt.style.use('fivethirtyeight')

#Surpress warnings
import warnings
warnings.simplefilter('ignore')

**Read in data**

In [149]:
#Set path to training data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Airbnb - San Francisco Listings Analysis\Data\03_Processed'

#Read data
df = pd.read_csv(path + '/2020_0608_Listings_Processed.csv', index_col=0)



**Preview Data**

In [150]:
#Preview data
print('Data shape:',df.shape)
display(df.head())

Data shape: (82433, 42)


,accommodates,amenities_count,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,cancellation_policy,cleaning_fee,extra_people,guests_included,holiday,host_has_profile_pic,host_identity_verified,host_is_superhost,host_response_rate,host_response_time,instant_bookable,is_location_exact,last_review_month_encoded,last_review_weekday_encoded,last_review_year_encoded,maximum_nights,minimum_nights,neighbourhood_cleansed_encoded,number_of_reviews,price,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_rating,reviews_per_month,room_type,security_deposit,weekend
0,3,36,25,106,43,58,1.000,Real Bed,1.000,2.000,1,0,0,moderate,100.000,25.000,2,False,1.000,1.000,1.000,90.000,within an hour,1,1,203.282,241.670,217.141,1125,1,248.227,240,170.000,Apartment,0,0,1,97.000,1.840,Entire home/apt,100.000,False
1,5,17,0,0,0,0,1.000,Real Bed,2.000,3.000,1,0,0,strict 14 with grace period,100.000,0.000,2,False,1.000,1.000,0.000,100.000,within a day,0,1,223.910,217.530,215.357,60,30,185.379,111,235.000,Apartment,0,0,1,98.000,0.830,Entire home/apt,2.000,True
2,2,18,30,365,60,90,4.000,Real Bed,1.000,1.000,9,9,0,strict 14 with grace period,50.000,12.000,1,False,1.000,1.000,0.000,100.000,within an hour,0,1,203.282,241.670,217.141,60,32,211.965,19,65.000,Apartment,0,0,1,84.000,0.150,Private room,200.000,False
3,2,16,30,365,60,90,4.000,Real Bed,1.000,1.000,9,9,0,strict 14 with grace period,50.000,12.000,1,False,1.000,1.000,0.000,100.000,within an hour,0,1,202.584,209.425,223.098,90,32,211.965,8,65.000,Apartment,0,0,1,93.000,0.120,Private room,200.000,False
5,3,33,30,173,56,84,1.000,Real Bed,1.000,1.000,2,2,0,strict 14 with grace period,50.000,60.000,2,False,1.000,0.000,1.000,100.000,within an hour,1,1,203.282,208.033,217.141,14,1,220.275,736,139.000,Condominium,0,0,1,98.000,5.660,Private room,0.000,False


# Model Development

### Prepare Data

**Split data into X and y**

In [151]:
#Split data into X and y
X = df.drop('price', axis = 1)
y = df.price.values

**One-Hot Encode**

In [153]:
#Read in Libraries
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#Subset categorical variables
cat_features = X.select_dtypes('object').columns

#Init ColumnTransformer 
ct = ColumnTransformer([('onehot', OneHotEncoder(handle_unknown='ignore'), cat_features)],remainder='passthrough')

#One-hot encode categorical variables'
X_encoded = ct.fit_transform(X)

**Split data and scale**

In [154]:
#Import libraries
from sklearn.model_selection import train_test_split

#Split data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, 
                                                    test_size = .2,
                                                   random_state = 24)

#Check
print('Training data:{} | Training labels:{}'.format(X_train.shape,y_train.shape))
print('Test data:{} | Test labels:{}'.format(X_test.shape,y_test.shape))

Training data:(65946, 85) | Training labels:(65946,)
Test data:(16487, 85) | Test labels:(16487,)


In [155]:
#Init scaler
scaler = StandardScaler()

#Fit and transform training and test data
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

## Base model

In [156]:
#Read in libraries
import tensorflow as tf
from tensorflow import keras

#Init base model
base_model = tf.keras.models.Sequential()

#Add Layers
base_model.add(tf.keras.layers.Dense(units = 64, activation = 'relu'))
base_model.add(tf.keras.layers.Dense(units = 64, activation = 'relu'))
base_model.add(tf.keras.layers.Dense(units = 1))

#compile
base_model.compile(loss = 'mean_squared_error', optimizer = 'Adam',)

In [157]:
#Train model
history = base_model.fit(X_train_transformed,y_train, batch_size= 100,epochs=200)

Epoch 1/200
660/660 [==============================] - 1s 1ms/step - loss: 8213.5469
Epoch 2/200
660/660 [==============================] - 1s 1ms/step - loss: 3456.1499
Epoch 3/200
660/660 [==============================] - 1s 1ms/step - loss: 3294.1404
Epoch 4/200
660/660 [==============================] - 1s 1ms/step - loss: 3193.3606
Epoch 5/200
660/660 [==============================] - 1s 1ms/step - loss: 3098.5898
Epoch 6/200
660/660 [==============================] - 1s 1ms/step - loss: 3009.0715
Epoch 7/200
660/660 [==============================] - 1s 1ms/step - loss: 2920.1843
Epoch 8/200
660/660 [==============================] - 1s 1ms/step - loss: 2839.7432
Epoch 9/200
660/660 [==============================] - 1s 1ms/step - loss: 2763.1677
Epoch 10/200
660/660 [==============================] - 1s 1ms/step - loss: 2695.1829
Epoch 11/200
660/660 [==============================] - 1s 1ms/step - loss: 2632.6082
Epoch 12/200
660/660 [==============================] - 1s 988u

**Evaluate Training Loss**

In [ ]:
#Store training loss as a data frame
training_loss = pd.DataFrame(history, columns='loss')

#Plot training loss
training_loss.plot()

**Evaluate base model on test data**

In [ ]:
#Evaluate performance on test data
predictions = 